In [1]:
import torchvision
import torch
import pandas as pd
import torch.fx as fx
import os

from model_static_graph import extract_graph, draw_graph
from pytorch_tracing import py_tracing_forward, py_tracing_backward, py_tracing_optimize
from op_kernel_dict import get_op_kernel


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)

#model
model = torchvision.models.resnet18().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
input_data = torch.randn(5, 3, 224, 224).cuda()
model_name='resnet18'

#静态图
static_graph,name_module,adj=extract_graph(model)
draw_graph(adj,name_module, model_name=model_name)
#draw_graph(adj,name_module, model_name=model_name,t=1)

#pytorch_tracing
py_tracing_forward(model, input_data, model_name=model_name)
py_tracing_backward(model, input_data, model_name=model_name)
py_tracing_optimize(model, input_data, optimizer, model_name=model_name)


/home/k8smaster/miniconda3/envs/k8s/lib/python3.10/site-packages/torch/profiler/profiler.py:445: UserWarning: Profiler won't be using warmup, this can skew profiler results
  warn("Profiler won't be using warmup, this can skew profiler results")


正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 已成功转化为 log_csv/resnet18_backward 目录下的csv
正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
正在处理文件 resnet18_optimize.json...
文件 resnet18_optimize.json 已成功转化为 log_csv/resnet18_optimize 目录下的csv
所有文件转换完成。
Profiling completed. Trace log saved to './log/resnet18_forward.json'
CSV files saved to 'log_csv'
正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 已成功转化为 log_csv/resnet18_backward 目录下的csv
正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
正在处理文件 resnet18_optimize.json...
文件 resnet18_optimize.json 已成功转化为 log_csv/resnet18_optimize 目录下的csv
所有文件转换完成。
Profiling completed. Trace log saved to './log/resnet18_backward.json'
CSV files saved to 'log_csv'
正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 

In [13]:
from VGG import VGG16

model = VGG16().cuda()
input_data = torch.randn(1, 3, 224, 224).cuda()
model_name='VGG16'
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

py_tracing_forward(model, input_data, model_name=model_name)
py_tracing_backward(model, input_data, model_name=model_name)
py_tracing_optimize(model, input_data, optimizer, model_name=model_name)

/home/k8smaster/miniconda3/envs/k8s/lib/python3.10/site-packages/torch/profiler/profiler.py:445: UserWarning: Profiler won't be using warmup, this can skew profiler results
  warn("Profiler won't be using warmup, this can skew profiler results")


正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 已成功转化为 log_csv/resnet18_backward 目录下的csv
正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 VGG16_forward.json...
文件 VGG16_forward.json 已成功转化为 log_csv/VGG16_forward 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
正在处理文件 resnet18_optimize.json...
文件 resnet18_optimize.json 已成功转化为 log_csv/resnet18_optimize 目录下的csv
所有文件转换完成。
Profiling completed. Trace log saved to './log/VGG16_forward.json'
CSV files saved to 'log_csv'
正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 已成功转化为 log_csv/resnet18_backward 目录下的csv
正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 VGG16_forward.json...
文件 VGG16_forward.json 已成功转化为 log_csv/VGG16_forward 目录下的csv
正在处理文件 VGG16_backward.json...
文件 VGG16_backward.json 已成功转化为 log_csv/VGG16_backward 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
正在处理文

In [2]:
from op_kernel_dict import get_op_kernel

Type = ['forward','backward','optimize']
model_name=['resnet18','VGG16']
#op_kernel={'resnet18':{},'VGG16':{}}
op_kernel={}
for model in model_name:
    for tp in Type:
        op_tmp=get_op_kernel(tp, model)
        #对于op_kernel 里没有的key 直接更新key value，对于已经存在的ker，计算op_tmp与op_kernel的并集
        op_kernel.update(op_tmp)



In [4]:
#比较op_kernel里每个key的set是否有交集
for key1 in op_kernel:
    for key2 in op_kernel:
        if key1!=key2:
            if len(op_kernel[key1]&op_kernel[key2])>0:
                print(key1,key2,op_kernel[key1]&op_kernel[key2])

#key的set
op_kernel

aten::cudnn_convolution aten::convolution_backward {'void cudnn::engines_precompiled::nchwToNhwcKernel<float, float, float, false, true, (cudnnKernelDataType_t)2>(cudnn::engines_precompiled::nchw2nhwc_params_t<float>, float const*, float*)', 'void cudnn::winograd::generateWinogradTilesKernel<0, float, float>(cudnn::winograd::GenerateWinogradTilesParams<float, float>)', 'void cudnn::engines_precompiled::nhwcToNchwKernel<float, float, float, true, false, (cudnnKernelDataType_t)0>(cudnn::engines_precompiled::nhwc2nchw_params_t<float>, float const*, float*)', '_5x_cudnn_ampere_scudnn_winograd_128x128_ldg1_ldg4_relu_tile148t_nt_v1'}
aten::clamp_min_ aten::clamp {'void at::native::vectorized_elementwise_kernel<4, at::native::(anonymous namespace)::launch_clamp_scalar(at::TensorIteratorBase&, c10::Scalar, c10::Scalar, at::native::detail::ClampLimits)::{lambda()#1}::operator()() const::{lambda()#7}::operator()() const::{lambda(float)#1}, at::detail::Array<char*, 2> >(int, at::native::(anonymou

{'aten::cudnn_convolution': {'_5x_cudnn_ampere_scudnn_128x64_relu_medium_nn_v1',
  '_5x_cudnn_ampere_scudnn_winograd_128x128_ldg1_ldg4_relu_tile148t_nt_v1',
  'sm86_xmma_fprop_implicit_gemm_indexed_tf32f32_tf32f32_f32_nhwckrsc_nhwc_tilesize128x64x32_stage4_warpsize2x2x1_g1_tensor16x8x8_execute_kernel__5x_cudnn',
  'sm86_xmma_fprop_implicit_gemm_tf32f32_tf32f32_f32_nhwckrsc_nchw_tilesize128x128x16_stage3_warpsize2x2x1_g1_tensor16x8x8_alignc4_execute_kernel__5x_cudnn',
  'sm86_xmma_fprop_implicit_gemm_tf32f32_tf32f32_f32_nhwckrsc_nchw_tilesize256x128x32_stage2_warpsize4x2x1_g1_tensor16x8x8_alignc4_execute_kernel__5x_cudnn',
  'void cask__5x_cudnn::computeOffsetsKernel<false, false>(cask__5x_cudnn::ComputeOffsetsParams)',
  'void cudnn::engines_precompiled::nchwToNhwcKernel<float, float, float, false, true, (cudnnKernelDataType_t)2>(cudnn::engines_precompiled::nchw2nhwc_params_t<float>, float const*, float*)',
  'void cudnn::engines_precompiled::nhwcToNchwKernel<float, float, float, true,